# Projet de DataMining :  

Dans ce projet nous allons recommander des images en fonctions d'images déjà "liké".
Pour cela nous avons commencé par traiter une base de 733 images en recupérant le maximun de données. On recupére notament les données Exif, la taille de l'image, son orientation, ect ...  
La base de données d'images vient du lien suivant:
https://www.kaggle.com/aksha05/flower-image-dataset
## Requirements : 

In [ ]:
from PIL import Image
from pandas import json_normalize
import pandas as pd
import json
import os

## Récupération des données et organisations de ces données dans un fichier json:

In [ ]:
path = "images/flowers"
dataJson = open('data.json', 'w')
dataTot = []
for file in os.listdir(path): #On peut lire l'ensemble des fichiers présents dans le dossier donné par path
    data = {}
    if file[0] != ".": #On exclue les fichiers "cachés"
        new_path = path + "/" + file
        image = Image.open(new_path) #Récupération de l'image avec le module Image de PIL
        tupleSize = image.size
        data["name"] = file[:-10]
        data["size"] = tupleSize
        data["format"] = image.format
        if tupleSize[0] > tupleSize[1] : 
            data["orientation"] = "paysage"
        elif tupleSize[0] < tupleSize[1] :
            data["orientation"] = "portrait"
        else:
            data["orientation"] = "carre"
        dataTot.append(data)

json.dump(dataTot,dataJson, indent = 4)
dataJson.close()
dataTot_frame = json_normalize(dataTot)
print(dataTot_frame)



In [ ]:
dataframe = pd.read_json('data.json')
print(dataframe)